<div style="text-align: center; font-weight: bold; font-size: 300%">EN6 MAP412</div>                                                        <br />      
<div style="text-align: center; font-size: 150%">École Polytechnique, MAP412 - 14 octobre 2019</div><br />  
<div style="text-align: center; font-size: 120%">Paul CALOT</div>

### Temps de travail: environ 10 heures.

In [1]:
import numpy as np

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import row, column

from numpy.linalg import solve, cond, inv, det, eig, qr, norm
from scipy.linalg import lu_factor, lu_solve, lu, solve_banded
output_notebook(hide_banner=True)

from time import time, sleep

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from bokeh.palettes import Viridis256
from bokeh.util.hex import hexbin

import pylab as pl

from scipy.sparse import csc_matrix
from scipy.sparse.linalg import splu

# Conditionnement

Considérons l'exemple suivant :
$$
    A=\begin{pmatrix}
      10 & 1 & 4 & 0 \\
      1  & 10& 5 & -1\\
      4 & 5 & 10 & 7 \\
      0 & -1 & 7 & 9
    \end{pmatrix}
    \text{ avec }
    b_1 = \begin{pmatrix}
    15\\15\\26\\15
    \end{pmatrix}
    \text{ et }
    b_2 = \begin{pmatrix}
    16\\16\\25\\16
    \end{pmatrix}
$$

**Calculez le déterminant de $A$ et l'inverse de $A$ avec les fonctions *numpy.linalg.inv* et *numpy.linalg.det***

In [2]:
# conditions initiales
A=np.array([[10,1,4,0],[1,10,5,-1],[4,5,10,7],[0,-1,7,9]])
b1=np.array([15,15,26,15])
b2=np.array([16,16,25,16])

# inverse et déterminant
Det=det(A)
Ainv=inv(A)

**Résolvez les systèmes $Ax_1=b_1$ et $Ax_2 = b_2$ en utilisant soit l'inverse trouvée ci-dessus, soit *numpy.linalg.solve*. Expliquez le comportement obtenu et donnez une borne inférieure pour le conditionnement de la matrice $A$.**

In [3]:
# solving
x1=solve(A,b1)
x2=solve(A,b2)
print("Solution du système Ax1=b1:", x1)
print("Solution du système Ax2=b2:", x2)


Solution du système Ax1=b1: [1. 1. 1. 1.]
Solution du système Ax2=b2: [  832.  1324. -2407.  2021.]


On a : $$ \kappa(A) \geq     \frac{\left\|\delta x\right\| \left\|b\right\|}{\left\|\delta b\right\|\left\|x\right\|} $$
Le calcul numérique donne ici:

In [4]:
KA=(norm(x2-x1)*norm(b1))/(norm(b2-b1)*norm(x1))
print("Le conditionnement de A est minoré par:", KA)

Le conditionnement de A est minoré par: 32252.894883611825


**Calculez le conditionnement de $A$ par les deux méthodes suivantes :**
* **calculez les valeurs propres de $A$ avec *numpy.linalg.eig* et utilisez le point précédent**
* **utiliser *numpy.linalg.cond***


In [5]:
# Par les valeurs propres
A=np.array([[10,1,4,0],[1,10,5,-1],[4,5,10,7],[0,-1,7,9]])
val=eig(A)[0]
Ka1=max(val)/min(val)
print("Le conditionnement de A est: ", Ka1)

Le conditionnement de A est:  35792.39762880096


On trouve un conditionnement de A cohérent avec la borne inférieure calculée précédemment.

In [6]:
# Par numy.linalg.cond
Ka2=cond(A) # on ne précise pas la norme qui est par défaut la norme 2.
print("Le conditionnement de A est: ", Ka2)

print("Différence entre les deux méthodes: ", abs(Ka2-Ka1))

Le conditionnement de A est:  35792.39762897281
Différence entre les deux méthodes:  1.7185084288939834e-07


On trouve un conditionnement de A cohérent avec la borne inférieure calculée avant qui n'est cependant pas identique à celle calculée précédemment. 

# Décomposition QR et résolution de systèmes surdimensionnés

On considère le système surdimensionné $Ax=b$, avec 
$$
    A=\begin{pmatrix}
      10 & 1 & 4  \\
      1  & 3 & 5 \\
      4 & 5  & 10 \\
      2 & 5  & 6 \\
      1 & 8  & 6 \\
      5 & 9  & -4 \\
      0 & -1 & 7 
    \end{pmatrix}\ 
    \text{ et }\ 
    b = \begin{pmatrix}
    14\\15\\26\\28\\47\\70 \\-15
    \end{pmatrix}
$$

**Utiliser la méthode de la question du sujet de PC pour résoudre le système linéaire surdimensionné. On pourra utiliser la fonction *numpy.linalg.qr*.**

Afficher la vraie solution.

In [7]:
A=np.array([[10,1,4],[1,3,5],[4,5,10],[2,5,6],[1,8,6],[5,9,-4],[0,-1,7]])
b=np.array([14,15,26,28,47,70,-15])

# décomposition Q, R
qrA=qr(A,'complete')
Q,R=qrA[0],qrA[1]
shapeR=np.shape(R) # comme R est une matrice triangulaire supérieure, 
                   # seules les i premières lignes nous intéressent, où i est le nombre de colonnes.
    # Résolution
Rint=[R[i] for i in range(shapeR[1])]
bint=np.dot(np.transpose(Q),b) # pb de shapes
bint2=[bint[i] for i in range(shapeR[1])]
Rint=[R[i] for i in range(shapeR[1])]

y=solve(Rint,bint2)
print("La solution du système linéaire surdimmensionné est: ", y) 


La solution du système linéaire surdimmensionné est:  [ 1.210814    6.58773664 -1.18569127]


# Stockage de matrice et Laplacien

 ## Cas 1D

 ### Matrice pleine
 
On considère une matrice $A$ tridiagonale, *i.e.* dont les seuls coefficients non-nuls sont donnés par :
$A_{i,j} \neq 0 \quad\Rightarrow\quad |i-j| \le 1.$  

On écrit cette matrice $A$ sous la forme
  $$ A = \left( \begin{array}{cccccc}
    a_1    & b_1    & 0       &\dots   & 0 \\
    c_2    & a_2    & b_2     & \ddots& \vdots  \\
    0      & \ddots & \ddots &\ddots  &0\\
    \vdots & \ddots & \ddots &\ddots  & b_{N-1}\\
    0      & \dots  &  0   &   c_N  & a_N
  \end{array}\right).$$
  
  
Pour les applications numériques, on fixera :
\begin{equation} a_i = \frac{-2}{h^2}, \quad b_i = \frac{1}{h^2} \quad \text{ et } c_i = \frac{1}{h^2} \text{ avec } h = \frac{1}{N+1}, \quad N = 2048.
\end{equation}

Ici $h$ est un pas d'espace et $N$ un nombre de mailles. Cette matrice correspond à une discrétisation de l'opérateur $-\Delta$, dont la construction sera vue plus tard dans le cours.

**On utilisera les valeurs numériques ci-dessus. Utiliser la fonction *scipy.linalg.lu_factor* pour construire deux matrices $L$ et $U$ respectivement triangulaire inférieure et supérieure telles que $A=LU$.**

In [8]:
# création de la matrice A

def A_Laplace(N):
    A=np.zeros((N,N))
    a=(N+1)*(N+1)
    for i in range(N-1):
        A[i][i]=-2*a
        A[i+1][i]=a
        A[i][i+1]=a
    A[N-1][N-1]=-2*a
    return A

N=2048
A=-A_Laplace(N) # on ajoute bien le "-"
LU=lu_factor(A)

**Résoudre $-Au=f$ avec $f_i = x_i^3$ où $x_i = ih$ pour $i=1,...,N$ en utilisant la factorisation LU précédente et la fonction *scipy.linalg.lu_solve*.**

In [9]:
def X(N):
    Xv=np.zeros(N)
    h=1/(N+1)
    Xv[0]=h
    for k in range(1,N):
        Xv[k]=h+Xv[k-1]
    return Xv


N=2048

Xx=X(N)
fx=Xx**3
A=-A_Laplace(N)
LU=lu_factor(A)
u=lu_solve(LU,fx)
#print(u)

**Calculer la solution analytique $x\mapsto u(x)$ du problème $-\Delta u = f$ et tracer les courbes $u(x_i)$ et $u_i$ en fonction de $x_i$.**

In [10]:
def u_analytique(x):
    return( x*(1-x*x*x*x)/20)


ua=u_analytique(Xx)
Err=abs(ua-u)

fig = figure(width=800, height=400,title="Solutions Laplacien 1D")
fig.x(Xx,u,legend="solution numérique",size=1)
fig.line(Xx,ua,legend="solution analytique",color="green",line_width=1)
fig.xaxis.axis_label = "xi"
fig.yaxis.axis_label = "u(x)"
fig.legend.location = "top_left"
show(fig)

fig1 = figure(width=800, height=400,title="Erreur commise - Laplacien 1D")
fig1.x(Xx,Err,size=1)
fig1.xaxis.axis_label = "xi"
fig1.yaxis.axis_label = "Erreur(xi)"
show(fig1)

### Matrice tridiagonale

**Implémenter l'algorithme de décomposition LU basé sur un stockage tridiagonal de la matrice, c'est-à-dire qu'on ne stocke que les composantes non-triviales des matrice $A$, $L$ et $U$. On écrira pour cela :**

\begin{gather}
      \tilde{A} = \left( \begin{array}{c|c|c|c} 0 & b_{1} & \dots & b_{N-1}\\ \hline  a_{1} & a_{2} & \dots & a_{N} \\ \hline c_{2} & c_{3} & \dots  & 0 \end{array} \right) = \left( \begin{array}{l} b \\ a \\ c \end{array}\right) \in \mathbb{R}^{3,N}, \nonumber \\
      \tilde{L} = \left( \gamma_{2}, \gamma_{3}, \dots, \gamma_{N}, 0 \right) \in\mathbb{R}^{1,N}, \qquad
      \tilde{U} = \left( \begin{array}{c|c|c|c|c} 0 & \beta_2 & \beta_3 & \dots & \beta_N \\ \hline \alpha_{1} & \alpha_{2} & \alpha_{3} & \dots & \alpha_{N} \end{array} \right) \in\mathbb{R}^{2,N}.
      %\tilde{A} = \left( \begin{array}{c|c|c} 0 & A_{1,1} & A_{1,2} \\ A_{2,1} & A_{2,2} & A_{2,3} \\ \vdots & \vdots & \vdots \\  A_{N,N-1} & A_{N,N} & 0 \end{array} \right) \in \mathbb{R}^{3,N}, \nonumber \\
      %\tilde{L} = \left( \begin{array}{c} 0 \\ L_{2,1} \\ L_{3,2} \\ \vdots \\ L_{N,N-1}\end{array}\right) \in\mathbb{R}^{N,1}, \qquad
      %\tilde{U} = \left( \begin{array}{c|c} U_{1,1} & U_{1,2} \\ \vdots & \vdots \\ U_{N-1,N-1} & U_{N-1,N} \\ U_{N,N} & 0\end{array} \right) \in\mathbb{R}^{N,2}.
        %= (0, \ A_{1,1},\ A_{1,2}), \quad \tilde{A}_{1,i} = (A_{i,i-1}, \ A_{i,i}, \ A_{i,i+1}), \quad \tilde{A}_{N,:} = (A_{N,N-1},\ A_{N,N}, \ 0).
\end{gather}

In [11]:
# on suppose qu'on dispose de A sous forme tridiagonale
def A_tri(A):
    N=np.shape(A)[0]
    At=np.zeros((3,N))
    for i in range(N-1):
        At[1][i]=A[i][i] # aj
        At[2][i]=A[i+1][i] # cj
        At[0][i+1]=A[i][i+1] # bj
    At[1][N-1]=A[N-1][N-1]
    return At

# On dispose maintenant de notre matrice en stockage tridiagonale. Construisons L et U.

def LU_tri(At):
    N=np.shape(At)[1]
    
    Lt=np.zeros(N)
    Ut=np.zeros((2,N))
    
    Ut[1][0]=At[1][0] # initialisation
    
    for i in range(1,N):
        Lt[i-1]=At[2][i-1]/Ut[1][i-1]
        Ut[0][i]=At[0][i]
        Ut[1][i]=At[1][i]-Lt[i-1]*Ut[0][i]
    
    return (Lt, Ut)




**Tester votre algorithme sur la matrice :**

\begin{equation} 
A = \left(\begin{array}{ccc} 2 & 1 & 0 \\ -1 & 3 & 1 \\ 0 & 1 & 4 \end{array}\right).
\end{equation}

In [12]:
A=np.array([[2,1,0],[-1,3,1],[0,1,4]])
P,L,U=lu(-A)

At=-A_tri(A) # dans ce cas, Atilde a autant de coefficient que A.

Lt,Ut=LU_tri(At) 

print("matrice L pleine :")
print(L)
print("matrice U pleine : ")
print(U)
print('')
print("matrice At, stockage tridiagonale : ")
print(At)
print("matrice Lt, stockage tridiagonale : ")
print(Lt)
print("matrice Ut, stockage tridiagonale : ")
print(Ut)



matrice L pleine :
[[ 1.          0.          0.        ]
 [-0.5         1.          0.        ]
 [-0.          0.28571429  1.        ]]
matrice U pleine : 
[[-2.         -1.          0.        ]
 [ 0.         -3.5        -1.        ]
 [ 0.          0.         -3.71428571]]

matrice At, stockage tridiagonale : 
[[-0. -1. -1.]
 [-2. -3. -4.]
 [ 1. -1. -0.]]
matrice Lt, stockage tridiagonale : 
[-0.5         0.28571429  0.        ]
matrice Ut, stockage tridiagonale : 
[[ 0.         -1.         -1.        ]
 [-2.         -3.5        -3.71428571]]


Pour une telle matrice, il n'y a pas vraiment d'avantages à fonctionner par stockage tridiagonal (la taille de la nouvelle matrice est toujours 3x3). On obtient les bons résultats en comparant à ce que renvoie $lu$ de $scipy.linalg$.

**Adapter et implémenter les algorithmes de remontée et de descente adaptés aux matrices triangulaires stockées sous forme tridiagonale. Tester votre algorithme pour la résolution de $- A u = f$ avec la matrice $A$ de question précedente et $f = (1,\ 1,\ 1)^T$, en utilisant la décomposition $LU$ de $A$ obtenue à la question précédente.**

In [13]:
def Descente_L(L,b): # résout Lx=b , b vecteur ligne
    n=len(b)
    X=np.zeros(n)
    
    # initialisation
    X[0]= b[0]
    # algorithme descendant
    for k in range(1,n):
        X[k]=b[k]-L[k-1]*X[k-1]
    
    return X

def Montee_U(U,b): # résout Ux=b
    n=len(b)
    X=np.zeros(n)
    
    # initialisation
    X[-1]= b[-1]/U[1][-1]
    
    #algorithme montant
    for i in range(n-2,-1,-1):
        X[i]=(b[i]-U[0][i+1]*X[i+1])/U[1][i]
    
    return X


In [31]:
A=np.array([[2,1,0],[-1,3,1],[0,1,4]])

At=-A_tri(A) # dans ce cas, Atilde a autant de coefficient que A.

Lt,Ut=LU_tri(At) 
fx_t=np.array([1,1,1]) 

# résolution stockage tridiagonal
u_int=Descente_L(Lt,fx_t)
u_final=Montee_U(Ut,u_int)

# résolution stockage normal par lu_solve
LU=lu_factor(-A)
fx=np.array([[1],[1],[1]])
u=lu_solve(LU,fx)

print("On obtient donc la solution:", u_final," pour un stockage tridiagonal.")
print("On obtient: ", u," pour un stockage classique.")


[[-0. -1. -1.]
 [-2. -3. -4.]
 [ 1. -1. -0.]]
On obtient donc la solution: [-0.30769231 -0.38461538 -0.15384615]  pour un stockage tridiagonal.
On obtient:  [[-0.30769231]
 [-0.38461538]
 [-0.15384615]]  pour un stockage classique.


On a bien les deux mêmes solutions.

**Résoudre le système $-Au = f$ avec la matrice $A$ issue de la discrétisation de l'opérateur $-\Delta$ . Comparer la précision des résultats et les temps d'execution (en utilisant *time.time*) avec ceux obtenus avec la matrice pleine. Commenter les avantages et inconvénients de ce stockage.**

In [36]:

# initialisation 
N=3
Xx=X(N)
fx=Xx**3 # attention au "-"
A=-A_Laplace(N)
# délai pour s'assurer que les calculs d'avant ne vont pas nuire à la mesure (processeur: taches effectées en série)
sleep(3)
#
    # cas 1:
t1=time()

LU=lu_factor(A)
u=lu_solve(LU,fx)

t2=time()
dt1=t2-t1

# délai pour s'assurer que les calculs d'avant ne vont pas nuire à la mesure (processeur: taches effectées en série)
sleep(3)
#
    # cas 2:
t1=time()

At=A_tri(A) # dans ce cas, Atilde a autant de coefficient que A.
Lt,Ut=LU_tri(At) 
u_int=Descente_L(Lt,fx)
u_final=Montee_U(Ut,u_int)
t2=time()
dt2=t2-t1


[0.01123047 0.02148438 0.02392578]


In [33]:
# résultat temps
print("Solution obtenue en ",dt1," secondes pour un stockage normal.")
print("Solution obtenue en ",dt2," secondes pour un stockage tridiagonal.")
print("Le stockage tridiagonal est ici ",dt1/dt2,"fois plus rapide que le stockage normal.")

# tracé
fig = figure(width=800, height=400,title="Solutions Laplacien 1D - Stockage tridiagonal")
fig.x(Xx,u,legend="cas normal",color="blue",size=2)
fig.x(Xx,u_final,legend="stockage tridiagonal",color="red",size=2)
fig.line(fx,u_analytique(fx),legend="solution analytique",color="green",line_width=1)
fig.legend.location = "top_left"
show(fig)



Solution obtenue en  0.1794130802154541  secondes pour un stockage normal.
Solution obtenue en  0.039754390716552734  secondes pour un stockage tridiagonal.
Le stockage tridiagonal est ici  4.5130381067757375 fois plus rapide que le stockage normal.


In [17]:
# tracé de l'erreur ua=u_analytique(Xx)
ua=u_analytique(Xx)
Err1=abs(ua-u)
Err2=abs(ua-u_final)
fig1 = figure(width=800, height=400,title="Erreur commise - Laplacien 1D")
fig1.dash(Xx,Err1,legend="Erreur stockage normal",color="red",line_width=1)
fig1.dash(Xx,Err2,legend="Erreur stockage tridiagonal",line_width=1)
fig1.xaxis.axis_label = "xi"
fig1.yaxis.axis_label = "Erreur(xi)"
fig1.legend.location = "top_left"

show(fig1)

Les avantages du stockage tridiagonal: les calculs sont plus rapides (facteur 14 ici), la précision semble la même. Cependant cette méthode demande des algorithmes à redéfinir selon le type de problèmes qu'on essaie de résoudre. En effet, on utilise les propriétés du Laplacien pour améliorer le stockage et la complexité des algorithmes en général. Notre méthode ne fonctionne donc que pour les systèmes de ce type. 

## Cas 2D

On considère maintenant une matrice bande, *i.e.* dont les coefficients satisfont si $|i-j| < K$ alors$A_{i,j} = 0,$ où $K>0$ est appelée la largeur de bande.

  On utilisera pour les applications numériques la matrice $A$ définie par :
  
\begin{align}
    A &= \left( \begin{array}{c|c|c|c}
      D    & I/h^2    &\dots   & 0 \\ \hline
      I/h^2    & D    & \ddots& \vdots  \\ \hline
      \vdots   & \ddots &\ddots  &I/h^2\\ \hline
      0      & \dots  &   I/h^2  & D
    \end{array}\right), \quad %\in\mathbb{R}^{N^2\times N^2}, \
    D = \left( \begin{array}{cccc}
      -4/h^2    & 1/h^2    & \dots   & 0 \\
      1/h^2    & -4/h^2    & \ddots& \vdots  \\
      0      & \ddots & \ddots  &0\\
      0      & \dots  &  1/h^2  & -4/h^2
    \end{array}\right) \in\mathbb{R}^{N\times N},
    \label{Lapl_2D}
\end{align}

où $I$ est la matrice identité de taille $N=64$ et $h = \frac{1}{N+1}$. Cette matrice correspond à une discrétisation de l'opérateur $-\Delta$ en 2D.

### Matrices pleines

**En utilisant la fonction *solve* de numpy, résoudre le système $-Au=f$ avec $f_{(i-1)N+j} = f(x_i,y_j) = x_i^3 + y_j^3$ où $x_i = ih$ et $y_j =jh$ pour $i,j=1,...,N$. Commenter la vitesse d'execution de l'algorithme et la taille de la matrice stockée (*sauvegarder avant d'executer*).**

In [18]:
# fonctionne aussi pour Y
def X(N):
    Xx=np.zeros(N)
    h=1/(N+1)
    x=h
    
    for k in range(N):
        Xx[k]=x
        x+=h
        
    return Xx

    # définition de f 
def f(X,Y,N):
    fx=np.zeros(N*N)
    
    for i in range(N):
        x=X[i]
        
        for j in range(N):
            y=Y[j]
            fx[i*N+j]=x*x*x+y*y*y
            
    return fx

        # définition de la matrice D
def D_mat(N): 
    v=((N+1)*(N+1)) * np.ones(N) # a priori très légèrement meilleure que de le faire en compréhension
    
    # en compréhension : 
    #a=1/((N+2)*(N+2))
    #v=np.array([a for k in range(N)])
    
    D=np.diag(-4*v)+np.diag(v[1:],k=1)+np.diag(v[1:],k=-1)  # utilisation de numpy.diag
    
    return D
        
        # définition de la matrice A
def A_mat(N):
    
    a=(N+1)*(N+1)
    IN=a*np.identity(N)
    A=np.zeros((N*N,N*N))
    D=D_mat(N)
    
    for k in range(N-1):
        A[k*N:(k+1)*N,k*N:(k+1)*N]=D
        A[k*N:(k+1)*N,(k+1)*N:(k+2)*N]=IN # diag inf
        A[(k+1)*N:(k+2)*N,k*N:(k+1)*N]=IN # diag sup
    A[(N-1)*N:N*N,(N-1)*N:N*N]=D
    
    return A

In [19]:
# en utilisant la définition par bloc
def A_matBis(N):
    
    a=(N+1)*(N+1)
    IN=a*np.identity(N)
    D=D_mat(N)
    O=np.zeros((N,N))
    # création des lignes de la Matrice A
    L=[]
    for i in range(0,N):
        l=[]
        if i == 0:
            l.append(D)
            l.append(IN)
            
            for k in range(N-2):
                l.append(O)
                
        elif i == N-1:
            for k in range(N-2):
                l.append(O)
            
            l.append(IN)
            l.append(D)

        else:
            for k in range(i-1):
                l.append(O)
            
            l.append(IN)
            l.append(D)
            l.append(IN)
            
            for k in range(N-2-i):
                l.append(O)
                
        L.append(l)

    A=np.block(L) # tentative de définition par block 
    return A


In [20]:
# test A_matBis(N) ou A_mat(N)
N=64

sleep(3)

t1=time()
D1=A_matBis(N) 
t2=time()

print("Temps mis pour une définition par bloc (A_matBis): ", t2-t1," secondes.")

sleep(3)

t1=time()
D2=A_mat(N)
t2=time()

print("Temps mis pour une définition par 'boucle for' (A_mat): ", t2-t1," secondes.")



Temps mis pour une définition par bloc (A_matBis):  0.15974950790405273  secondes.
Temps mis pour une définition par 'boucle for' (A_mat):  0.024437665939331055  secondes.


Je ne prendrais pas ce résultat comme étant absolu. En effet, j'ai utilisé dans $A\_matBis$ des listes pythons, par facilité, que j'ai transformé à la fin en $array$ $numpy$. Il aurait mieux valu travailler uniquement avec des $array$ $numpy$.

In [21]:
carac=[] # va contenir les caractéristiques pour chaque test

In [22]:
N=64

sleep(3)

t0=time()

    # définition de la matrice -A
A=-A_mat(N)


ti1=time()
    # défintion de f:
Xx=X(N)

ti2=time()

Yx=np.copy(Xx)

ti3=time()

fx=f(Xx,Yx,N)

t1=time()

    # Résolution du problème:
Uplein=solve(A,fx)

t2=time()


dt1=t2-t1
dt0=t1-t0
s=np.shape(A)
    
print("temps de définition de A : ", ti1-t0," secondes")
print("temps de définition de Xx : ", ti2-ti1," secondes")
print("temps de définition de Yx : ", ti3-ti2," secondes")
print("temps de définition de fx : ", t1-ti3," secondes")

print("")
print("")

t=s[0]*s[1]*64/10**6
carac.append([dt1,dt0,t])
# affichage
print("La taille de la matrice A est: ", s," soit ",s[0]*s[1]," éléments à stocker, soit sur 64 bits: ",t," Mo.")
print("Le temps de résolution est: ",dt1," secondes.")
print("Le temps d'affectations des variables avant la résolution est : ",dt0," secondes.")

temps de définition de A :  0.20989251136779785  secondes
temps de définition de Xx :  0.0  secondes
temps de définition de Yx :  0.0  secondes
temps de définition de fx :  0.005004167556762695  secondes


La taille de la matrice A est:  (4096, 4096)  soit  16777216  éléments à stocker, soit sur 64 bits:  1073.741824  Mo.
Le temps de résolution est:  0.678138256072998  secondes.
Le temps d'affectations des variables avant la résolution est :  0.21489667892456055  secondes.


L'espace de stockage de plus d' $1$ $Go$ et les temps de calculs élevés ne sont pas viables pour plein d'appliquations, et on est uniquement en 2D! En effet, il est impossible d'utiliser cela si on veut résoudre ce système pour représenter en temps réel un quelconque phénomène physique en 2D et à 30 images par secondes par exemple (on simplifierait probablement aussi la représentation).

### Matrices bandes

On stocke désormais la matrice sous forme bande, c'est-à-dire qu'on écrit :

\begin{align*}
      \tilde{A} = \left(
      \begin{array}{c|c|c|c|c|c}
        0             & \dots & 0        & A_{1,K+1} & \dots   & A_{N-K,N}\\
        \hline \vdots &       &  & A_{2,K+1} &\dots   & A_{N-K+1,N}\\
        \hline 0      & A_{1,2} &          &          &         & \vdots    \\
        \hline A_{1,1} & A_{2,2} & \dots  & \dots     & \dots   & A_{N,N} \\
        \hline A_{2,1} & A_{3,2} & \dots   & \dots    & A_{N,N-1} & 0 \\
        \hline \vdots &        &         &          &         & \vdots \\
        \hline A_{K+1,1} & \dots & A_{N,N-K} & 0       & \dots   &  0
      \end{array} \right) \in \mathbb{R}^{2K+1,N}.
\end{align*}

**Utiliser la fonction *solve_banded* de scipy exploitant cette structure bande pour résoudre le problème $-Au = f$. Comparer la vitesse d'execution et la taille de la matrice utilisée avec la question précédente.**

In [23]:
def A_tri2D(N): # à revoir. C'est faux. La matrice qu'on obtient pour N=3 n'est pas la bonne.
    N2=N*N
    a=(N+1)*(N+1)
    At=np.zeros((2*N+1,N2))
    
    for i in range(0,N2-N):
        At[N][i]=-4*a            # diagonale milieu
        if not ((i+1)%N==0): 
            At[N-1][i+1]=a       # diagonale supérieure
            At[N+1][i]=a         # diagonale inférieure
        At[0][i+N]=a             # diagonale la plus haute 
        At[2*N][i]=a             # diagonale la plus basse
    
    for i in range(0,N-1):
        At[N][i+N2-N]=-4*a
        At[N-1][i+1+N2-N]=a
        At[N+1][i+N2-N]=a

        
    At[N][N2-1]=-4*a
    
    return At

In [24]:
# Test
N=3

print("La matrice sous stockage tridiagonale: ")
print("")
print(A_tri2D(N))
print("")
print("Doit correspondre sous sa forme pleine à :")
print("")
print(A_mat(N))

print("")
print("C'est bon ! ")

La matrice sous stockage tridiagonale: 

[[  0.   0.   0.  16.  16.  16.  16.  16.  16.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.  16.  16.   0.  16.  16.   0.  16.  16.]
 [-64. -64. -64. -64. -64. -64. -64. -64. -64.]
 [ 16.  16.   0.  16.  16.   0.  16.  16.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ 16.  16.  16.  16.  16.  16.   0.   0.   0.]]

Doit correspondre sous sa forme pleine à :

[[-64.  16.   0.  16.   0.   0.   0.   0.   0.]
 [ 16. -64.  16.   0.  16.   0.   0.   0.   0.]
 [  0.  16. -64.   0.   0.  16.   0.   0.   0.]
 [ 16.   0.   0. -64.  16.   0.  16.   0.   0.]
 [  0.  16.   0.  16. -64.  16.   0.  16.   0.]
 [  0.   0.  16.   0.  16. -64.   0.   0.  16.]
 [  0.   0.   0.  16.   0.   0. -64.  16.   0.]
 [  0.   0.   0.   0.  16.   0.  16. -64.  16.]
 [  0.   0.   0.   0.   0.  16.   0.  16. -64.]]

C'est bon ! 


In [25]:
N=64

sleep(3)

t0=time()

# définition de la matrice -A
A=-A_tri2D(N)

# défintion de f:
Xx=X(N)
Yx=np.copy(Xx)

fx=f(Xx,Yx,N)

t1=time()

# Résolution du problème:
Utri=solve_banded((N,N),A,fx)

t2=time()
dt1=t2-t1
dt0=t1-t0

s=np.shape(A)

t=s[0]*s[1]*64/10**6
carac.append([dt1,dt0,t])
print("La taille de la matrice A sous format bandes est: ", s," soit ",s[0]*s[1]," éléments à stocker, soit sur 64 bits: ",s[0]*s[1]*64/10**6," Mo.")
print("Le temps de résolution est: ",dt1," secondes.")
print("Le temps d'affectation avant résolution est: ",dt0," secondes.")

La taille de la matrice A sous format bandes est:  (129, 4096)  soit  528384  éléments à stocker, soit sur 64 bits:  33.816576  Mo.
Le temps de résolution est:  0.02521371841430664  secondes.
Le temps d'affectation avant résolution est:  0.05958843231201172  secondes.


A la fois l'affectation et la résolution sont plus rapides (respectivement 10 et 40 fois plus rapides environ). Cependant, on reste toujours trop élevé pour espérer faire de la simulation en temps réel à 30 images par secondes par exemple (ips).

### Matrices creuses CSR

**Bonus. Construire la matrice $A$ de la discrétisation de l'opérateur $-\Delta$ en format CSR (utiliser scipy.sparse.csr matrix) avec Scipy (https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html).**

In [26]:
# matrice dense:
N=64

# Définition de la matrice par ses lignes et colonne.
# on pourrait encore légèrement améliorer: ici on stocke toujours les "0" explicites placé sur la diagonale.
def CSC_A(N):
    N2=N*N
    a=(N+1)*(N+1)
    L=np.zeros((3,5*N2-2*N-2))
    for k in range(N2): # diagonale du milieu
        L[0][k]=k # ligne
        L[1][k]=k # colonne
        L[2][k]=-4*a
        
    for k in range(N2-1): # diagonale supérieure intérieur
        L[0][N2+k]=k # ligne 
        L[1][N2+k]=k+1 # colonne
        if not ((k+1)%N==0):
            L[2][N2+k]=a
        
    for k in range(N2-1): # diagonale inférieure intérieur$
        L[0][2*N2-1+k]=k+1 # ligne
        L[1][2*N2-1+k]=k # colonne
        if not ((k+1)%N==0):
            L[2][2*N2-1+k]=a # vaut 0 sinon
    
    for k in range(N2-N): # diagonale supérieure extérieure
        L[0][3*N2-2+k]=k # ligne
        L[1][3*N2-2+k]=k+N # colonne
        L[2][3*N2-2+k]=a
        
    for k in range(N2-N): # diagonale inférieur extérieure
        L[0][4*N2-N-2+k]=k+N # ligne
        L[1][4*N2-N-2+k]=k # colonne
        L[2][4*N2-N-2+k]=a
        
    return csc_matrix((L[2],(L[0],L[1])),[N2,N2])

In [27]:
# vérification 
N=64
Acsc=CSC_A(N)
A=A_mat(N)
Acsc2=csc_matrix(A)

A1=Acsc.toarray()
A2=Acsc2.toarray()
print(A1-A2) # ok

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


**Bonus. Résoudre le système $-Au = f$ avec la fonction *scipy.sparse.linalg.splu* et comparer la vitesse d’exécution et la taille de la matrice utilisée avec les deux méthodes précédentes.**

In [28]:
N=64

sleep(3)

t0=time()

    # définition des objets utilisés
#A=A_mat(N)
#Acsc=csc_matrix(A)
Acsc=CSC_A(N)

#At=splu(Acsc.sorted_indices())
At=splu(-Acsc)  # lu decomposition # attention au "-"

    # définition de f
Xx=X(N)
Yx=np.copy(Xx)
fx=f(Xx,Yx,N)

t1=time()

# résolution 
Ucsc=At.solve(fx)

t2=time()

s=Acsc.nnz # nombre d'éléments stockés (nombre d'éléments non nuls ou zéros explicites )
print("Le nombre d'éléments explicits est: ",s)
print("Le nombre d'éléments non nuls est: ", 5*N*N-4*N) # le nombre d'éléments non nuls (on a enlevé les 0 explicits)

assert s==(5*N*N-2*N-2) # on vérifie que le nombre d'éléments explicits est bien en accord avec la théorie

dt1=t2-t1
dt0=t1-t0

t=3*s*64/10**6
carac.append([dt1,dt0,t])
print("")
print("La taille de la matrice Acsr est: ", s," soit ",3*s," éléments à stocker, soit sur 64 bits: ",3*s*64/10**6," Mo.")
print("Le temps de résolution est: ",dt1," secondes.")
print("Le temps d'affectation avant la résolution est: ",dt0," secondes.")

Le nombre d'éléments explicits est:  20350
Le nombre d'éléments non nuls est:  20224

La taille de la matrice Acsr est:  20350  soit  61050  éléments à stocker, soit sur 64 bits:  3.9072  Mo.
Le temps de résolution est:  0.0  secondes.
Le temps d'affectation avant la résolution est:  0.06009340286254883  secondes.


### Erreur maximale de la différence entre les solutions de chacune de ces méthodes: 

In [29]:
print("Différence entre Pleine - Bande: ", (max(abs(Uplein-Utri))))
print("Différence entre Pleµine - CSR : ", (max(abs(Uplein-Ucsc))))
print("Différence entre CSR - Bande : ", (max(abs(Ucsc-Utri))))

Différence entre Pleine - Bande:  2.914335439641036e-16
Différence entre Pleµine - CSR :  2.185751579730777e-16
Différence entre CSR - Bande :  3.5735303605122226e-16


Les résultats s'accordent, les erreurs sont de l'ordre de l'erreur machine.

## Conclusion:

In [30]:
print("Matrice pleine:")
print("Temps de résolution pur : ", carac[0][0], "secondes  ;  temps d'affectation: ", carac[0][1],"secondes  ;   taille :", carac[0][2], "Mo.")

print("")

print("Matrice Bande:")
print("Temps de résolution pur : ", carac[1][0], "secondes  ;  temps d'affectation: ", carac[1][1],"secondes  ;  taille :", carac[1][2], "Mo.")

print("")

print("Matrice Creuse:")
print("Temps de résolution pur : ", carac[2][0], "secondes  ;  temps d'affectation: ", carac[2][1],"secondes  ;  taille :", carac[2][2], "Mo.")

Matrice pleine:
Temps de résolution pur :  0.678138256072998 secondes  ;  temps d'affectation:  0.21489667892456055 secondes  ;   taille : 1073.741824 Mo.

Matrice Bande:
Temps de résolution pur :  0.02521371841430664 secondes  ;  temps d'affectation:  0.05958843231201172 secondes  ;  taille : 33.816576 Mo.

Matrice Creuse:
Temps de résolution pur :  0.0 secondes  ;  temps d'affectation:  0.06009340286254883 secondes  ;  taille : 3.9072 Mo.


Le stockage en matrice creuse est encore plus efficace pour la résolution que celui par bande. Du point de vue de la résolution seule, on peut espérer simuler en temps réel (30 ips) un phénomène physique par exemple. Cependant le temps d'affectation initial est assez long, principalement dû à la création de la matrice A. Ainsi, si on résout toujours la même équation, on peut stocker A une fois pour toutes (à N fixé) et simplement changer les conditions limites qui elles sont modifiées après chaque pas de temps. 